# Flight Price Forecast - Model Evaluation

This notebook provides comprehensive evaluation of trained models:

- Assess model performance using multiple regression metrics
- Create confusion matrices and performance visualizations
- Detect overfitting and underfitting issues
- Compare model performance across different scenarios
- Generate detailed evaluation reports

**Prerequisites**: Complete model training in `04_model_training.ipynb`.


## 1. Load Trained Models and Data


## 2. Comprehensive Model Evaluation
